# Level 2 - Week 4 - 04 Chat Endpoint Contract

**Estimated time:** 60-90 minutes

## Learning Objectives

- Define chat request and response schemas
- Include citations and mode
- Keep /chat thin and testable


## Overview

A thin /chat endpoint keeps retrieval and validation testable.
Define a stable contract early.

## Practice Steps

- Create ChatRequest and ChatResponse models.
- Add a build_chat_response helper.


### Sample code

Chat request and response schema.


In [ ]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

class ChatRequest(BaseModel):
    question: str = Field(min_length=1)
    top_k: int = Field(default=5, ge=1, le=50)
    filters: Dict | None = None

class Citation(BaseModel):
    doc_id: str
    chunk_id: str
    snippet: str

class ChatResponse(BaseModel):
    answer: Optional[str] = None
    citations: List[Citation] = []
    mode: str


### Student fill-in

Build a response payload based on mode and citations.


In [ ]:
def build_chat_response(answer: str | None, citations: list[dict], mode: str) -> dict:
    return {'answer': answer, 'citations': citations, 'mode': mode}

# TODO: add validation of mode and citations


## Self-check

- Is mode always one of answer, clarify, refuse?
- Are citations traceable?
